# Language modelling

The exercise shows how a language model may be used to solve word-prediction tasks and to generate text.


## Tasks

1. Read the documentation of [Language modelling in the Transformers](https://huggingface.co/transformers/task_summary.html#language-modeling) library.
2. Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository. 

3. Produce the predictions for the following sentences (use each model and check 5 predictions):


    i. (M) Warszawa to największe `[MASK]`.
    ii. (D) Te zabawki należą do `[MASK]`.
    iii. (C) Policjant przygląda się `[MASK]`.
    iv. (B) Na środku skrzyżowania widać `[MASK]`.
    v. (N) Właściciel samochodu widział złodzieja z `[MASK]`.
    vi. (Ms) Prezydent z premierem rozmawiali wczoraj o `[MASK]`.
    vii. (W) Witaj drogi `[MASK]`.

In [2]:
from transformers import AutoTokenizer, AutoModel



In [3]:
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = AutoModel.from_pretrained("allegro/herbert-base-cased")

output = model(
    **tokenizer.batch_encode_plus(
        [
            (
                "A potem szedł środkiem drogi w kurzawie, bo zamiatał nogami, ślepy dziad prowadzony przez tłustego kundla na sznurku.",
                "A potem leciał od lasu chłopak z butelką, ale ten ujrzawszy księdza przy drodze okrążył go z dala i biegł na przełaj pól do karczmy."
            )
        ],
    padding='longest',
    add_special_tokens=True,
    return_tensors='pt'
    )
)

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from pprint import pprint
from transformers import * 
pprint(unmasker(f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."))


AttributeError: module transformers.models.cpm has no attribute CpmTokenizer

In [8]:
import inspect
import transformers
[m[0] for m in inspect.getmembers(transformers, inspect.isclass) if m[1].__module__ == 'unmasker']


[]

In [ ]:
import pprint

from transformers import pipeline
unmasker = pipeline("fill-mask")
pprint(unmasker(f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."))


In [14]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased")

sequence = f"Warszawa to największe {tokenizer.mask_token}."
def predict(sequence):
    inputs = tokenizer(sequence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    token_logits = model(**inputs).logits

    mask_token_logits = token_logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
    for token in top_5_tokens:
        print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:

predict(f"Warszawa to największe {tokenizer.mask_token}.")
predict(f"Te zabawki należą do {tokenizer.mask_token}.")
predict(f"Policjant przygląda się {tokenizer.mask_token}.")
predict(f"Policjant przygląda się {tokenizer.mask_token}.")
predict(f"Na środku skrzyżowania widać {tokenizer.mask_token}.")
predict(f"Właściciel samochodu widział złodzieja z {tokenizer.mask_token}.")
predict(f"Prezydent z premierem rozmawiali wczoraj o {tokenizer.mask_token}.")
predict(f"Witaj drogi {tokenizer.mask_token}.")


Warszawa to największe miasto.
Warszawa to największe lotnisko.
Warszawa to największe centrum.
Warszawa to największe miasta.
Warszawa to największe atrakcje.
Te zabawki należą do rodziny.
Te zabawki należą do nas.
Te zabawki należą do nich.
Te zabawki należą do najlepszych.
Te zabawki należą do ..
Policjant przygląda się mężczyźnie.
Policjant przygląda się kobiecie.
Policjant przygląda się mu.
Policjant przygląda się dziewczynie.
Policjant przygląda się sprawie.
Policjant przygląda się mężczyźnie.
Policjant przygląda się kobiecie.
Policjant przygląda się mu.
Policjant przygląda się dziewczynie.
Policjant przygląda się sprawie.
Na środku skrzyżowania widać rondo.
Na środku skrzyżowania widać samochody.
Na środku skrzyżowania widać radiowóz.
Na środku skrzyżowania widać samochód.
Na środku skrzyżowania widać wiadukt.
Właściciel samochodu widział złodzieja z samochodu.
Właściciel samochodu widział złodzieja z włamaniem.
Właściciel samochodu widział złodzieja z auta.
Właściciel samochodu

   
4. Check the model predictions for the following sentences (using each model):


    i. Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie `[MASK]`.
    ii. Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie `[MASK]`.
   

In [19]:
predict(f"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie {tokenizer.mask_token}.")
predict(f"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie {tokenizer.mask_token}.")


Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie poddał.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zdziwił.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie dowiedział.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zastanawiał.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie przyznał.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie dowiedziała.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie przyznała.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie bała.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zmieniła.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zgodziła.


5. Check the model predictions for the following sentences:


    i. `[MASK]` wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
    ii. W wakacje odwiedziłem `[MASK]`, który jest stolicą Islandii.
    iii. Informatyka na `[MASK]` należy do najlepszych kierunków w Polsce.
   

6. If you want to use causal language models such as PapuGaPT2 or plT5, you should change the last three examples to accomodate for the fact, that these
   models are better suited for causal language modelling.
   

7. Answer the following questions:
   
   
    i. Which of the models produced the best results?
    ii. Was any of the models able to capture Polish grammar?
    iii. Was any of the models able to capture long-distant relationships between the words?
    iv. Was any of the models able to capture world knowledge?
    v. What are the most striking errors made by the models?